# Exploratory Analysis of a Monthly Dataset

In [8]:
# do the below to load the python utilities from py_depends

In [9]:
cd ..

/Users/svyatnakonechny


In [10]:
import os
from py_depends.utils import *
from aux.dtypes import *

# visualizations and such
import seaborn

# sanity check - assert the import worked correctly
try:
    assert 'flight-delays' in os.getcwd()
except AssertionError:
    print('Doesn\'t look like the cd .. hack worked - restart the kernel and try again')

Doesn't look like the cd .. hack worked - restart the kernel and try again


_First, some housekeeping. Reading in a sample file,_

In [11]:
%time jan_2018_df = unzipFile('./sample_data/On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2018_1.zip')
# and assigning correct(?) types
%time subs_jan_2018_df = subsetRenameRetype(jan_2018_df, new_cols)
# and finally shaping those date-time columns - this is a massive bottleneck because it loops through the entire dataframe _for each column pair_. Incredibly inefficient
%time subs_jan_2018_df = shapeDatetime(subs_jan_2018_df, ['crs_dep_time', 'dep_time','crs_arr_time', 'arr_time', 'first_dep_time', 'diverted_wheels_off_1', 'diverted_wheels_off_2', 'diverted_wheels_off_3'], 'flight_date')

FileNotFoundError: [Errno 2] No such file or directory: './sample_data/On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2018_1.zip'

CPU times: user 3.85 s, sys: 467 ms, total: 4.31 s
Wall time: 4.32 s
Shaping crs_dep_time-flight_date columns
Shaping dep_time-flight_date columns
Shaping crs_arr_time-flight_date columns
Shaping arr_time-flight_date columns
Shaping first_dep_time-flight_date columns
Shaping diverted_wheels_off_1-flight_date columns
Shaping diverted_wheels_off_2-flight_date columns
Shaping diverted_wheels_off_3-flight_date columns
CPU times: user 3min 28s, sys: 5.93 s, total: 3min 34s
Wall time: 3min 36s


## Descriptive stats

### Airports

In [76]:
departure_delay_stats = subs_jan_2018_df.groupby('origin')['dep_delay'].agg(['count', 'mean']).reset_index()
counts_late_departure = subs_jan_2018_df[subs_jan_2018_df.dep_delay > 0].groupby('origin')['origin'].agg(['count']).reset_index()
departure_delay_stats.columns = ['origin', 'total_flights', 'mean_delay']
counts_late_departure.columns = ['origin', 'delayed_flights']
departure_delay_stats = pd.merge(departure_delay_stats, counts_late_departure, how='left')
departure_delay_stats['percentage_departed_late'] = departure_delay_stats['delayed_flights']/departure_delay_stats['total_flights']
departure_delay_stats.sort_values(by='total_flights', ascending=False).head(15)

,origin,total_flights,mean_delay,delayed_flights,percentage_departed_late
20,ATL,29836,11.858828,11016.0,0.369218
241,ORD,29270,15.037444,11072.0,0.378271
88,DFW,21962,6.920044,6738.0,0.306803
87,DEN,20247,9.555342,6841.0,0.337877
184,LAX,20110,5.865092,5779.0,0.287369
69,CLT,18782,8.731232,6553.0,0.348898
255,PHX,14597,8.285333,5838.0,0.399945
303,SFO,14447,7.745553,4388.0,0.303731
301,SEA,14127,2.275288,3451.0,0.244284
94,DTW,13565,14.429266,4655.0,0.343163


Top 15 airports by departures - no massive surprises. Lots of departures from Atlanta, Chicago O'Hare, and the likes, with mean delays between 5 and 15 mins. Looks like the busier an airport is the more likely it is to experiece departure delays (and then there's NYC airports).

In [73]:
departure_delay_stats

,origin,count,mean,delayed_flights
0,ABE,387,15.847545,141.0
1,ABI,166,11.463855,55.0
2,ABQ,2140,5.283645,581.0
3,ABR,59,16.457627,20.0
4,ABY,76,13.368421,20.0
...,...,...,...,...
346,XNA,1049,12.977121,279.0
347,YAK,59,-22.135593,5.0
348,YKM,115,3.582609,30.0
349,YNG,2,63.000000,2.0


In [54]:
len(departure_delay_stats)

351